In [ ]:
from ultralytics import YOLO
from wandb.integration.ultralytics import add_wandb_callback
import os  
import wandb
import yaml
import warnings 
warnings.filterwarnings(action='ignore', category=UserWarning)

In [ ]:
# Initialize WandB
project="human_parsing"

wandb.init(project=project)

# Load a model
pretrained = "weights/yolov8n-seg.pt"
pretrained = "/home/ubuntu/JD-HD/SEGMENT/human_parsing/train16/weights/best.pt"
model = YOLO(pretrained, task='segment')  # Load a pretrained model (recommended for training)

# Load labels from YAML configuration file
yaml_file = "configs/fashion_people_detection.yml"
with open(yaml_file, 'r') as f:
    config = yaml.safe_load(f)
    class_labels = config['names']  # Adjust the key based on your YAML structure

# Add WandB callback
add_wandb_callback(model)

bs = 72
workers = os.cpu_count()
# Train the model
results = model.train(project=project, data=yaml_file, epochs=100, imgsz=1024, batch=bs, workers=workers)

# Step 5: Validate the Model
model.val()

# Finish the W&B run
wandb.finish()

In [ ]:
import random
from PIL import Image
import requests
from utils import *
import yaml

yaml_file = "configs/fashion_people_detection.yml"
with open(yaml_file, 'r') as f:
    config = yaml.safe_load(f)

labels_dict = config.get('names')

In [ ]:
model_path = "/home/ubuntu/JD-HD/SEGMENT/human_parsing/train17/weights/best.pt"
model = YOLO(model_path, task="segment")

In [ ]:
url = "https://www.the-sun.com/wp-content/uploads/sites/6/2023/09/janice-dickinson-attends-36th-carousel-805923729.jpg"
image = Image.open(requests.get(url, stream=True).raw)
image = resize_preserve_aspect_ratio(image, 1024)
results = model(image)

In [ ]:
result = results[0]
labels = result.boxes.cls 
labels = [int(label.item()) for label in labels]
labels = [labels_dict[label] for label in labels]
masks = result.masks.data
masks = [unload_mask(mask) for mask in masks]

masks_dict = dict(zip(labels, masks))
for k, v in masks_dict.items():
    print(k)
    overlay = overlay_mask(image, v)
    display(overlay)